In [ ]:
!pip install  langchain
!pip install  langchain_community
!pip install  pypdf -U
!pip install  transformers==4.38.2
!pip install  tokenizers==0.15.2
!pip install  sentence-transformers
!pip install  elasticsearch
!pip install  langchain_elasticsearch
!pip install  rapidocr-onnxruntime
!pip install  langchain_cohere
!pip install  cohere


#Import dependencies
from cohere import Client
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import warnings
warnings.filterwarnings(action='ignore')
from threading import Thread
from typing import Optional
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_elasticsearch import ElasticsearchStore
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms.base import LLM
from langchain_community.llms import Cohere
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from elasticsearch import Elasticsearch
from datetime import datetime
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def load_document(data_path):
  loader = PyPDFDirectoryLoader(path = data_path, glob='**/[!.]*.pdf', extract_images=True)
  documents = loader.load()
  return documents

def split_document(documents):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
  chunks = text_splitter.split_documents(documents)  #split documents into chunks
  return chunks

In [ ]:
docs = load_document("/content/drive/My Drive/Série principale des lettres de Darby/") #chemin vers les lettres de Darby sur Drive

In [ ]:
docs = split_document(docs)

In [ ]:
print(len(docs))

4998


In [ ]:
docs[586].page_content

'moi je ne les ai pas comptés. De plus, tous les invités sont venus et avant tout\nCelui qui ne fait jamais défaut aux besoins et aux faibles désirs de ses enfants. Il\nnous a bénis de sa bonté; les frères étaient tout à fait au lar ge et heureux entre eux.\nIl y en avait deux qui n’étaient pas formellement des nôtres; l’un, ministre\nnational, qui a reçu, je le crois, beaucoup de bien, et qui probablement n’avait\njamais rien vu de semblable. Nous avons été en paix dans ce pauvre coin de'

In [ ]:
user_name = "gnoseia"
ELASTIC_PASSWORD = "Virginie972!"
CLOUD_ID = "GnoseIA:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ5OWJiZjIwNGVmN2Q0MDY4YTMyMjdlN2Y3OTQyZjQxNCQ5ZWFkNjNjYjExM2M0MTA2OWExOGNhYmQxZWQxYmE1Zg=="

In [ ]:
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=(user_name, ELASTIC_PASSWORD)
)

In [ ]:
client.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': '99bbf204ef7d4068a3227e7f7942f414', 'cluster_uuid': 'GKYlFKbTSKS7UBHnaQ7lBQ', 'version': {'number': '8.13.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'da95df118650b55a500dcc181889ac35c6d8da7c', 'build_date': '2024-05-06T22:04:45.107454559Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
#generer un index pour les documents de chaque utilisateur en fonction du temps
def generate_index_name(user_name):
    """Génère un nom d'index basé sur la date actuelle."""
    current_date = datetime.now().strftime("%Y%m%d%H%M%S")
    index_name = f"{user_name.lower()}{current_date}"
    return index_name

In [ ]:
index_pattern = generate_index_name(user_name)    #index généré
print(index_pattern)

gnoseia20240530201458


In [ ]:
# Récupérer le contenu actuel du rôle
role_name = "enterprise-search-app-search-admin"
role = client.security.get_role(name=role_name, ignore=404)
role_body = role[role_name]
if index_pattern not in role_body['indices'][0]['names']:

   # Ajouter le nouvel index à la liste des index du rôle
   role_body['indices'][0]['names'].append(index_pattern)
   # Mettre à jour le rôle avec le nouvel index
   client.security.put_role(name=role_name, body= role_body)
print(role_body)

{'cluster': ['all', 'manage', 'monitor'], 'indices': [{'names': ['gnoseia', 'gnoseia20240528080141', 'gnoseia20240528083453', 'gnoseia20240529084718', 'gnoseia20240530201458'], 'privileges': ['all'], 'field_security': {'grant': ['*']}, 'allow_restricted_indices': False}], 'applications': [{'application': 'kibana-.kibana', 'privileges': ['feature_advancedSettings.read'], 'resources': ['*']}], 'run_as': ['gnoseia'], 'metadata': {}, 'transient_metadata': {'enabled': True}}


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_database = ElasticsearchStore.from_documents(
                documents=docs,
                es_cloud_id=CLOUD_ID,
                index_name=index_pattern,
                embedding=embeddings,
                es_user=user_name,
                es_password=ELASTIC_PASSWORD,
                distance_strategy="COSINE",
                strategy=ElasticsearchStore.ApproxRetrievalStrategy()
            )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
retriever = vector_database.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 30,                      #nombre des résultats tirés
        "score_threshold": 0.7        #seuil de la recherche, on prend les top 30 des phrases similaires à la question avec un seuil de 70%
    }
)

In [ ]:
cohere_client = Client(api_key='iqcG53EsoG2Ps2nq1eHZzhJeoNQ4NwR8XliswG9i')
compressor = CohereRerank(cohere_api_key='iqcG53EsoG2Ps2nq1eHZzhJeoNQ4NwR8XliswG9i', top_n = 5) #on prend les top 5 de ces top 30
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
# fonction pour initialiser tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    #tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [ ]:
# Function for streaming llm ouput word by word and reduce perceived latency
def init_model(model, tokenizer):
    class CustomLLM(LLM):

        streamer: Optional[TextIteratorStreamer] = None

        def _call(self, prompt, stop=None, run_manager=None) -> str:
            model.to("cuda")
            # nouveau prompt de notre LLM
            prompt_template = """Use the following context elements to answer the question at the end.
            If there is no given context or the context does not contain any relevant information for the question,
            simply say that you do not know, do not attempt to invent an answer.
            Answer like you were the author of the given context when answering the question.
            Always respond in the language in which the question is asked."""
            combined_prompt = f"{prompt_template}\n{prompt}"
            self.streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, timeout=5)
            inputs = tokenizer(combined_prompt, return_tensors="pt").to("cuda")
            kwargs = dict(input_ids=inputs["input_ids"],
                          streamer=self.streamer,
                          max_new_tokens=500)

            # Générer le texte et récupérer la sortie générée
            generated_ids = model.generate(**kwargs)
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            if combined_prompt in generated_text:
                generated_text = generated_text.replace(combined_prompt, '').strip()

            return generated_text.strip()

        @property
        def _llm_type(self) -> str:
            return "custom"

    llm = CustomLLM()
    return llm

In [ ]:
# Initilialiser tokenizer & model
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = initialize_tokenizer(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id
# specifier stop token ids
stop_token_ids = [0]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
llm=init_model(model,tokenizer)     #LLM personnalisé pour générer la reponse de l'IA

In [ ]:
def init_model_for_overview(model, tokenizer):
    class CustomLLM(LLM):

        streamer: Optional[TextIteratorStreamer] = None

        def _call(self, prompt, stop=None, run_manager=None) -> str:
            model.to("cuda")
            self.streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, timeout=5)
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            kwargs = dict(input_ids=inputs["input_ids"],
                          streamer=self.streamer,
                          max_new_tokens=512)

            # Générer le texte et récupérer la sortie générée
            generated_ids = model.generate(**kwargs)
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            if prompt in generated_text:
                generated_text = generated_text.replace(prompt, '').strip()

            return generated_text.strip()

        @property
        def _llm_type(self) -> str:
            return "custom"

    llm = CustomLLM()
    return llm
llm_for_overview = init_model_for_overview(model, tokenizer)   #LLM personnalisé pour l'aperçu général du document

NameError: name 'model' is not defined

In [ ]:
def combined_chunk(document:list)-> str:
    #doc = " ".join(chunk.page_content for chunk in document)
    doc = " ".join(document[i].page_content for i in range(0, 20))
    return doc
# Créer un modèle LangChain pour générer un aperçu général
def create_overview():
    prompt_template = """
        En utilisant le document ci-dessous, générez un texte illustrant
        son titre s'il contient un titre, le brève résumé de ce qu'il raconte et
        une proposition de 3 questions auxquelles ce document peut répondre. Essayez de toujours générer des phrases complètes dans le texte généré.
        Votre réponse doit être dans un seul et unique paragraphe et répondez en langue avec laquelle le document est écrit.


        {text}

        """
    prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
    #overview_chain = LLMChain(prompt=prompt, llm=llm_for_overview)
    overview_chain = ( prompt | llm_for_overview )
    return overview_chain
def generate_overview(document:str, overview_chain:LLMChain)-> str:
    overview = overview_chain.invoke({"text": document})
    return overview
def overview_chain(document:list)-> str:
  combined_document = combined_chunk(document)
  overview_chain = create_overview()
  overview = generate_overview(combined_document, overview_chain)
  return overview

In [ ]:
# build conversational retrieval chain with memory (rag) using langchain
def Response_IA(query: str, chat_history: list)-> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False,
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=compression_retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )
        result = qa_chain({'question': query, 'chat_history': chat_history})
        generated_text = result['answer']
        generated_text = generated_text.strip()
        chat_history.append((query, generated_text))

        return generated_text, chat_history

    except Exception as e:
        chat_history.append((query, e))
        return e, chat_history

In [ ]:
def reference_for_retriever(results):                   #fonction pour obtenir les références
  nombre_reference = 1
  for result in results:
    print(nombre_reference,")","'", result.page_content,"'")
    split = result.metadata["source"]
    splitted = split.split("/")
    source = splitted[-1]
    print("Source: ","'",source,"'", "dans la page", result.metadata["page"]+1)
    nombre_reference += 1

In [ ]:
vector_database.client.indices.refresh(index=index_pattern)

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

In [ ]:
question = "Que pense Darby des menteurs ?" # question à poser venant de l'utilisateur et récupérer à partir de front du site GnoseIA

In [ ]:
chat_history = []                                                            # initialisation de chat_history
reponseIA, chat_history = Response_IA(question, chat_history)
print(reponseIA)

name 'ConversationBufferMemory' is not defined


In [ ]:
results = compression_retriever.invoke(question)
reference_for_retriever(results)

1 ) ' 14/02/2024 20:17 Lettre de J.N.D. no 326 – Le Messager Evangélique
https://lemessagerevangelique.ch/1904/lettre-de-j-n-d-no-326/ 1/2
Lettre de J.N.D. no 326
Darby J.N. , année 1904 , page 218
   à Mr A.G., à Clairac
Nîmes, 25 avril 1850
Bien cher frère,
 Je vous écris un mot pour vous dire que j’ai renoncé à ma visite dans le Lot et
Garonne, avec l’espoir d’en faire une plus tard. L ’oeuvre en France grandit
évidemment, et j’ai pris la résolution de partir tout de suite pour la Suisse et, au '
Source:  ' Lettre de J.N.D. no 326 – Le Messager Evangélique.pdf ' dans la page 1
2 ) ' Depuis que je vous ai vu, j’ai parcouru le midi de la France; la Suisse française,
sauf la Vallée de Joux et le Pays d’En-haut; la Suisse allemande; Guebwiller;
quelques visites en passant à Amiens, etc., enfin le nord de l’Angleterre pendant
trois semaines, après une conférence où j’avais trouvé les portes ouvertes et un
travail très intéressant, puis quelques autres endroits. En ce moment, je suis à
E

In [ ]:
overview = overview_chain(docs)
print(overview)

Ce document s'intitule "Lettre de J.N.D. no 1" et dans ce document, l'auteur parle de la pratique de la sacrificature. Voici quelques questions que ce document peut répondre:
        
        1. Qu'est-ce que la sacrificature et comment s'exerce-t-elle?
        
        2. Quelle est la relation entre la sacrificature et la justice de Dieu en Christ?
        
        3. Quelle est la manière dont la sacrificature s'exerce pour nous maintenir en rapport avec la pleine afection et la sainteté du Père et de Dieu, selon la position dans laquelle Christ, notre Chef, se trouve?
        
        4. Qu'est-ce que la responsabilité des chrétiens et comment est-elle liée à nos privilèges?
        
        5. Qu'est-ce que la pratique des sacrifices pour le péché et comment est-elle liée à la justice de Dieu en Christ?
        
        Le texte explique que la sacrificature s'exerce en grâce pour nous, non sur notre demande, avant que nous ne demandions rien. Il affirme que la sacrificature de Ch